# Predictive Analysis - Performance Score (Stint Score)

In this notebook, we conduct machine learning analysis to predict a player's stint_score, henceforth referred to as their performance score.

In [13]:
import pandas as pd

df = pd.read_csv('Data/Gold/main.csv')
X = df[['playerID', 'stint_score']]

print(X)

        playerID  stint_score
0      abdelju01    -9.118580
1      abdelju01     7.683405
2      abdelju01     1.435175
3       abelcl01     7.648768
4       abelcl01    -1.878769
...          ...          ...
24954  zyuzian01     2.243539
24955  zyuzian01     5.757446
24956  zyuzian01     3.890254
24957  zyuzian01   -10.530467
24958  zyuzian01    -5.163647

[24959 rows x 2 columns]


## Model 1 - Random Forest?

In [14]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define your features and target variable
# Replace 'target_column' with the name of your target column
target_column = 'stint_score'
X = df[['age', 'teammates_same_nationality']]
y = df[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Feature importance
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
print("Feature Importance:")
print(feature_importance.sort_values(ascending=False))

# Function to make a single prediction with dynamic input values
def predict_single(input_values):
    # Ensure input_values is a DataFrame with the same columns as X
    input_df = pd.DataFrame([input_values], columns=X.columns)
    prediction = model.predict(input_df)
    return prediction[0]

# # Example usage of the predict_single function
# example_input = { 'feature1': value1, 'feature2': value2} # replace with actual feature names and values
# prediction = predict_single(example_input)
# print(f"Prediction for example input: {prediction}")


Mean Squared Error: 26.438465106456306
R-squared: 0.05212829920587925
Feature Importance:
age                           0.728802
teammates_same_nationality    0.271198
dtype: float64


## Model 2 - AdaBoost

## Compare Performance

## Hyperparameter Experimentation

## Summary